## Librerías

In [1]:
!pip install keras==2.9

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install tensorflow==2.9

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf
print(tf.__version__)

2024-01-29 21:25:36.896333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-29 21:25:36.896425: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.0


In [4]:
import sofa
import polars as pl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf
import os

# Scikit-Learn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.model_selection import KFold
# SciPy
from scipy.io import loadmat

# Tensorflow
from tensorflow.keras import models, regularizers, utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import time


# Funciones

## Demodulation Single Channel

In [5]:
def demodulation_single(X_rx, X_tx, spacing,lista_neuronas, OSNR,n_splits,activations, database):
    """
    Demodulación para single channel

    :param X_rx: Diccionario con los datos recibidos (por espaciamiento)
    :param X_tx: Datos recibidos (Se lee previamente)
    :spacing: Espaciamiento actual (para escribir en archivo)
    :OSNR: Lista con los valores de OSNR para calcular su respectivo BER
    :n_splits: Numero de kFolds para la red
    :activations: Funciones de Activación para las capas de la red
    """
    folder = "/home/jupyter-santiago.vargas5/DNN_Demodulation/results"
    #Variar el OSNR
    for i, snr in enumerate(X_rx):#Cambiar OSNR por X_rx si se desea calcular con todos los valores
        #snr=str(snr)+"dB"
        # Extraer información
        if snr not in database["single_ch"].keys():
            database["single_ch"][snr] = {}
            print(f"=================Iniciando {snr}=================")
        else:
            print(f"=================Continuando {snr}=================")
        #X_ch_norm = X_rx[snr].get("const_Y").flatten()
        X_ch_norm = np.array(X_rx[snr]['I'])+np.array(X_rx[snr]['Q'])*1j
        X_ch = sofa.mod_norm(X_ch_norm, 10) * X_ch_norm

        for neuron in lista_neuronas:
            if  (str(neuron)) not in database["single_ch"][snr].keys():
                DNN_BER_lista = []
                print(f"***********RED NEURONAL***********\n{neuron} Neuronas")
                # Sincronizar de las señales
                synced_X_tx = sofa.sync_signals(X_tx,  X_ch)[0]
                # Demodular señal transmitida
                y = sofa.demodulate(synced_X_tx, sofa.MOD_DICT)

                #Inicio asignacion de parametros para la red
                max_neurons = neuron
                layer_props_lst = [
                    {"units": max_neurons // (2**i), "activation": activation}
                    for i, activation in enumerate(activations)
                ]
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
                # Demodulación para redes
                t_inic = time.time()
                dem_DNN = sofa.demodulate_neural(
                    X_ch,
                    y,
                    layer_props_lst=layer_props_lst,
                    loss_fn=loss_fn,
                    n_splits = n_splits,
                    )
                demodulated, tests = dem_DNN
                #Calcular BER para cada KFold
                for demod, test in zip(demodulated, tests):
                    x_ch_dnn = np.argmax(demod, axis = 1)
                    k_BER = sofa.bit_error_rate(x_ch_dnn, test)
                    DNN_BER_lista.append(k_BER)
                print(f"Lista BER DNN: {DNN_BER_lista}")
                DNN_BER = np.mean(np.array(DNN_BER_lista))
                print(f"DNN mean BER=> {DNN_BER}")
                t_fin = time.time()
                print(f"Tiempo DNN: {t_fin - t_inic}\n")

                database["single_ch"][snr][neuron] = DNN_BER
                sofa.save_json(database, folder)
                print(f"Calculo con {neuron} NEURONAS terminado para OSNR {snr}dB ")
        print("Se han recorrido todas las neuronas")
    return database

## Demodulation General

In [6]:
def demodulation(X_rx, X_tx, spacing,lista_neuronas, OSNR,n_splits,activations, database, especific):
    """
    Demodulación con DNN para los casos que no son single channel

    :param X_rx: Diccionario con los datos recibidos (por espaciamiento)
    :param X_tx: Datos recibidos (Se lee previamente)
    :spacing: Espaciamiento actual (para escribir en archivo)
    :OSNR: Lista con los valores de OSNR para calcular su respectivo BER
    :n_splits: Numero de kFolds para la red
    :activations: Funciones de Activación para las capas de la red
    :database: Resultados en la base de datos
    :especific: Seleccionar snr específicos de la lista OSNR (true). Seleccionar todos
    """
    l_OSNR = X_rx
    if especific:
      l_OSNR = OSNR

    folder = "/home/jupyter-santiago.vargas5/DNN_Demodulation/results"
    #Variar el OSNR
    for i, snr in enumerate(l_OSNR):#Cambiar OSNR por X_rx si se desea calcular con todos los valores
        # Extraer información
        if especific: #En database los datos vienen con una etiqueta específica
          snr=str(snr)+"dB"
        print(snr)
        if snr not in database[f"{spacing}GHz"].keys():
            database[f"{spacing}GHz"][snr] = {}
            print(f"=================Iniciando {snr}=================")
        else:
            print(f"=================Continuando {snr}=================")

        X_ch_norm = np.array(X_rx[snr]['I'])+np.array(X_rx[snr]['Q'])*1j
        X_ch = sofa.mod_norm(X_ch_norm, 10) * X_ch_norm
        sufix=""
        for act in activations:#Etiquetar por funcion de activación
          sufix=sufix+act[0]
        print(sufix)
        for neuron in lista_neuronas:
            if  (sufix+str(neuron)) not in database[f"{spacing}GHz"][snr].keys():
                DNN_BER_lista = []
                print(f"***********RED NEURONAL***********\n{neuron} Neuronas")
                # Sincronizar de las señales
                synced_X_tx = sofa.sync_signals(X_tx,  X_ch)[0]
                # Demodular señal transmitida
                y = sofa.demodulate(synced_X_tx, sofa.MOD_DICT)

                #Inicio asignacion de parametros para la red
                max_neurons = neuron
                layer_props_lst = [
                    {"units": max_neurons // (2**i), "activation": activation}
                    for i, activation in enumerate(activations)
                ]
                #Función de pérdida
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
                t_inic = time.time()#Medir el tiempo
                dem_DNN = sofa.demodulate_neural(
                    X_ch,
                    y,
                    layer_props_lst=layer_props_lst,
                    loss_fn=loss_fn,
                    n_splits = n_splits,
                    )
                demodulated, tests = dem_DNN
                #Calcular BER para cada KFold
                for demod, test in zip(demodulated, tests):
                    x_ch_dnn = np.argmax(demod, axis = 1)
                    k_BER = sofa.bit_error_rate(x_ch_dnn, test)
                    DNN_BER_lista.append(k_BER)

                print(f"Lista BER DNN: {DNN_BER_lista}")
                #Promedio del los BER encontrados
                DNN_BER = np.mean(np.array(DNN_BER_lista))
                print(f"DNN mean BER=> {DNN_BER}")
                t_fin = time.time()
                print(f"Tiempo DNN: {t_fin - t_inic}\n")

                database[f"{spacing}GHz"][snr][f"{sufix}{neuron}"] = DNN_BER
                sofa.save_json(database, folder)
                print(f"Calculo con {neuron} NEURONAS terminado para OSNR {snr}")
        print("Se han recorrido todas las neuronas")
    return database

## Read Data

In [7]:
# Función especial para leer todos los datos con la estructura estudiada
def read_data(folder_rx):
    data = {}

    # Leer la carpeta principal
    for folder in os.listdir(folder_rx):
        # Leer las subcarpetas
        if folder.endswith("spacing"):
            data[folder] = {}
            for file in os.listdir(f"{folder_rx}/{folder}"):
                if file.find("consY") != -1:
                    data_name = file.split("_")[2]
                    if data[folder].get(data_name) == None:
                        data[folder][data_name] = {}
                    mat_file_data = loadmat(f"{folder_rx}/{folder}/{file}")
                    data[folder][data_name] = mat_file_data
    return data
# Special function to read the known data structure
def read_data(folder_rx, ends):
    data = {}

    # Read root directory
    for folder in os.listdir(folder_rx):
        # Check name consistency for subdirectories
        if folder.endswith(ends):
            # Extract "pretty" part of the name
            spacing = folder[:-8]
            data[spacing] = {}

            # Read each data file
            for file in os.listdir(f"{folder_rx}/{folder}"):
                # Check name consistency for data files
                if file.find("consY") != -1:
                    # Extract "pretty" part of the name
                    osnr = file.split("_")[2][5:-4]

                    # Initialize if not created yet
                    if data[spacing].get(osnr) == None:
                        data[spacing][osnr] = {}
                    # Set data
                    csv_file_data = pl.read_csv(f"{folder_rx}/{folder}/{file}")
                    data[spacing][osnr] = csv_file_data
    return data


# Load Data

In [9]:
folder_rx = "/home/jupyter-santiago.vargas5/DNN_Demodulation/data"
file_tx ="/home/jupyter-santiago.vargas5/DNN_Demodulation/data/2x16QAM_16GBd.csv"
folder_json = "/home/jupyter-santiago.vargas5/DNN_Demodulation/results"
# Datos transmitidos
X_tx_norm = pl.read_csv(file_tx)
X_tx_norm = np.array(X_tx_norm['I']) + np.array(X_tx_norm['Q'])*1j
X_tx = sofa.mod_norm(X_tx_norm, 10)*X_tx_norm
# Leer los datos recibidos
data = read_data(folder_rx,'spacing')
database = sofa.load_json(folder_json)

In [10]:
# folder = "/home/jupyter-santiago.vargas5/DNN_Demodulation/results"
# sofa.save_json(database, folder,3)

# Calcular DNN

[ ] Implementar opción para seleccionar modo:
- Seleccionar más capas.
- Seleccionar un espaciamiento en específico.

## Revisar Resultados y Posibles Errores

### Elementos Duplicados

In [10]:
def elementos_duplicados(diccionario):
    valores_vistos = {}
    duplicados = []

    for clave, valor in diccionario.items():
        if valor in valores_vistos:
            #print(f'REPETIDO {clave}')
            duplicados.append((clave, valor))
        else:
            valores_vistos[valor] = clave

    return duplicados

spacings = [15.5,16,16.5,17,17.6,18]
dic_repetidos = {}
for spacing in spacings:
    
    for osnr in database[f'{spacing}GHz']:
        
        if len(elementos_duplicados(database[f'{spacing}GHz'][osnr]))>0:
            #print(spacing, osnr)
            #print(elementos_duplicados(database[f'{spacing}GHz'][osnr]))
            resultado = elementos_duplicados(database[f'{spacing}GHz'][osnr])
            config_list = []
            for elementos in resultado:
                repet = list(database[f'{spacing}GHz'][osnr].keys())[list(database[f'{spacing}GHz'][osnr].values()).index(elementos[1])]
                config_list.append(repet)
                config_list.append(elementos[0])
                
                #print('Clave repetida: '+list(database[f'{spacing}GHz'][osnr].keys())[list(database[f'{spacing}GHz'][osnr].values()).index(elementos[1])])
                print('-------------------')
            dic_repetidos[f'{spacing}GHz'] = {}
            dic_repetidos[f'{spacing}GHz'][osnr] =  config_list
print(dic_repetidos)

-------------------
-------------------
-------------------
-------------------
-------------------
{'15.5GHz': {'27dB': ['s8', 'rr32']}, '17GHz': {'25dB': ['rs32', 'rts32', 'rtt50', 'rst50']}}


### Revisar Escenarios Pendientes

In [11]:
spacings = [15,15.5,16,16.5,17,17.6,18]
conf_list = ["rr32"]
for spacing in spacings:    
    for osnr in database[f'{spacing}GHz']:
        
        if not(set(conf_list) <= set(list(database[f'{spacing}GHz'][osnr].keys()))):
            resta = set(conf_list) - set(list(database[f'{spacing}GHz'][osnr].keys()))
            print(f"Revisar {spacing}GHz - {osnr} - {resta}")

Revisar 15.5GHz - 35dB - {'rr32'}
Revisar 16GHz - 23dB - {'rr32'}
Revisar 16GHz - 19dB - {'rr32'}
Revisar 16GHz - 20dB - {'rr32'}
Revisar 16GHz - 21.5dB - {'rr32'}
Revisar 16GHz - 18dB - {'rr32'}
Revisar 16GHz - 27dB - {'rr32'}
Revisar 16GHz - 30dB - {'rr32'}
Revisar 16GHz - 35dB - {'rr32'}
Revisar 16GHz - 25dB - {'rr32'}
Revisar 16GHz - 32dB - {'rr32'}
Revisar 16GHz - 40dB - {'rr32'}
Revisar 16.5GHz - 18dB - {'rr32'}
Revisar 16.5GHz - 20dB - {'rr32'}
Revisar 16.5GHz - 23dB - {'rr32'}
Revisar 16.5GHz - 21.5dB - {'rr32'}
Revisar 16.5GHz - 19dB - {'rr32'}
Revisar 16.5GHz - 35dB - {'rr32'}
Revisar 16.5GHz - 25dB - {'rr32'}
Revisar 16.5GHz - 30dB - {'rr32'}
Revisar 16.5GHz - 40dB - {'rr32'}
Revisar 16.5GHz - 32dB - {'rr32'}
Revisar 16.5GHz - 27dB - {'rr32'}
Revisar 17GHz - 18dB - {'rr32'}
Revisar 17GHz - 19dB - {'rr32'}
Revisar 17GHz - 20dB - {'rr32'}
Revisar 17GHz - 32dB - {'rr32'}
Revisar 17GHz - 35dB - {'rr32'}
Revisar 17GHz - 21.5dB - {'rr32'}
Revisar 17GHz - 25dB - {'rr32'}
Revisar 17

### Eliminar Elementos Duplicados

In [20]:
# for sp in dic_repetidos:
#     for snr in dic_repetidos[sp]:
#         for val_rep in dic_repetidos[sp][snr]:
#             del database[sp][snr][val_rep]

## Correr Modelo

In [ ]:
spacings = [15,15.5,16,16.5,17,17.6,18]
for spacing in spacings:
    X_rx = data[f'{spacing}GHz']
    print(f"Espaciamiento actual: {spacing}GHz")
    #demodulation(X_rx,X_tx, spacing,lista neuronas,[],4, ["relu","relu", "relu"],database, False)
    demodulation(X_rx=X_rx,
                 X_tx=X_tx,
                 spacing=spacing,
                 lista_neuronas=[32],
                 #Agregar OSNR específicos si se desea. (poner especific=True)
                 OSNR=[32],
                 n_splits=4,
                 activations=["relu", "relu"],
                 database=database,
                 #Indicar si se trabajan con valores de OSNR específico para evitar errores
                 especific=False
                 )

Espaciamiento actual: 15GHz
40dB
=================Continuando 40dB=================
rr
Se han recorrido todas las neuronas
30dB
=================Continuando 30dB=================
rr
Se han recorrido todas las neuronas
27dB
=================Continuando 27dB=================
rr
Se han recorrido todas las neuronas
32dB
=================Continuando 32dB=================
rr
Se han recorrido todas las neuronas
23dB
=================Continuando 23dB=================
rr
Se han recorrido todas las neuronas
25dB
=================Continuando 25dB=================
rr
Se han recorrido todas las neuronas
35dB
=================Continuando 35dB=================
rr
Se han recorrido todas las neuronas
Espaciamiento actual: 15.5GHz
40dB
=================Continuando 40dB=================
rr
Se han recorrido todas las neuronas
30dB
=================Continuando 30dB=================
rr
Se han recorrido todas las neuronas
27dB
=================Continuando 27dB=================
rr
Se han recorrido todas las n

In [ ]:
spacings = [15,15.5,16,16.5,17, 17.6,18]
for spacing in spacings:
    X_rx = data[f'{spacing}GHz']
    print(f"Espaciamiento actual: {spacing}GHz")
    #demodulation(X_rx,X_tx, spacing,lista neuronas,[],4, ["relu","relu", "relu"],database, False)
    demodulation(X_rx=X_rx,
                 X_tx=X_tx,
                 spacing=spacing,
                 lista_neuronas=[32],
                 #Agregar OSNR específicos si se desea. (poner especific=True)
                 OSNR=[32],
                 n_splits=4,
                 activations=["relu", "relu"],
                 database=database,
                 #Indicar si se trabajan con valores de OSNR específico para evitar errores
                 especific=False
                 )

### Combinaciones faltantes:
sigmoid 8 ---- DONE :D

tanh 8 --- In progress

relu 32, relu 16 --- In progress :D

sigmoid 32, relu 16
